# R Hartmann6

This notebook is an example of a single objective R model optimization using a Hartmann6 function. The Hartmann6 function is a synthetic function that is often used as a stand in for a model that is expensive to evaluate. The function has 6 dimensions and is defined on the unit hypercube.

See the folder `r_branin` for the R script and the configuration file. 

In [ ]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts

In [2]:
import pathlib

from IPython.display import Code

boa_paper_dir = pathlib.Path().resolve().parent
r_dir = boa_paper_dir / "r_saasbo_branin"
config_path = r_dir / "config.yaml"
run_model_script = r_dir / "run_model.R"
hartmann6_script = r_dir / "hartmann6.R"

## Configuration File Overview

config.yaml

In [4]:
Code(config_path)

optimization_options:
    objective_options:
        objectives:
            - name: metric
    experiment:
        name: "r_streamlined_run"
    trials: 15

parameters:
    x0:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x1:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'
    x2:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x3:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'
    x4:
        'bounds': [ 0, 1 ]
        'type': 'range'
        'value_type': 'float'
    x5:
        'bounds': [ 0, 1]
        'type': 'range'
        'value_type': 'float'

script_options:
    # notice here that this is a shell command
    # this is what BOA will do to launch your script
    # it will also pass as a command line argument the current trial directory
    # that is being parameterized

    # This can either be a relative path or absolute path
    # (by default when BOA launches from a config file
    # it uses the config file directory as your working directory)
    # here config.yaml and run_model.R are in the same directory
    run_model: Rscript run_model.R


# options only needed by the model and not BOA
# You can put anything here that your model might need
# We don't need anything extra so we leave it commented out
# model_options:
    # the_question: 42

## Run Model Wrapper Script

run_model.R

In [5]:
Code(run_model_script)

# load in any libraries and modules we need
library(jsonlite)
source("../r_utils/hartman6.R")

# This is where we read in from BOA the command line argument.
# If in your script, you use any other command line arguments,
# generally BOA's trial_dir should be the last command line arugment,
# so taking the last one should generally be safe.
args <- commandArgs(trailingOnly=TRUE)
trial_dir <- args[length(args)]

# this this trial_dir folder there are 2 files supplied by BOA,
# a parameters.json that has just the parameters, and a trial.json
# that includes the parameters and a lot more in case you need it.
# Most people will only need the parameters.json
param_path <- file.path(trial_dir, "parameters.json")
data <- read_json(path=param_path)

# The parameter keys config from whatever you  named them in your
# config file, which you are free to change.
x0 <- data$x0
x1 <- data$x1
x2 <- data$x2
x3 <- data$x3
x4 <- data$x4
x5 <- data$x5
X <- c(x0, x1, x2, x3, x4, x5)

# This is where we actually run our "model".
# Here we are using a synthetic function called hartman6
# But you could substitute it for your own model in
# a number of ways.
res <- hartman6(X)

# In this case, we directly ran the model, so we are getting back a number
# or nan, so we know if it succeeded or failed. If you are submitting a job
# to an HPC (a super computer) queue, this might work, or you might have to
# rely on another method. Other options could be relying on log file output
# or information from querying the queue itself,
# though those may be better as stand alone `Set Trial Status Scripts`
if (!is.na(res)) {

    # if it was a success, we don't even need to write out trial status,
    # it is assumed a success if we write out data and don't fail
    out_data <- list(
        metric=res
        # trial_status=unbox("COMPLETED")  #  this is optional if it succeeds
    )

} else {

    # If we fail, then we do need to include a trial status, and mark it as failed.
    out_data <- list(
        trial_status=unbox("FAILED")
    )
}

json_data <- toJSON(out_data, pretty = TRUE)
write(json_data, file.path(trial_dir, "output.json"))

We also use a function called hartman6 which is a 6 dimensional version of the synthetic hartman function as the stand in for our model function. The code is below. You would substitute this for any call your model, be it local call to your own R model, a system call to a fortran model wrapped in your R script, or perhaps a some code that launches an HPC job and collects the results.

hartmann6.R

In [5]:
Code(hartmann6_script)

# load in any libraries and modules we need
library(jsonlite)
source("../r_utils/hartman6.R")

# This is where we read in from BOA the command line argument.
# If in your script, you use any other command line arguments,
# generally BOA's trial_dir should be the last command line arugment,
# so taking the last one should generally be safe.
args <- commandArgs(trailingOnly=TRUE)
trial_dir <- args[length(args)]

# this this trial_dir folder there are 2 files supplied by BOA,
# a parameters.json that has just the parameters, and a trial.json
# that includes the parameters and a lot more in case you need it.
# Most people will only need the parameters.json
param_path <- file.path(trial_dir, "parameters.json")
data <- read_json(path=param_path)

# The parameter keys config from whatever you  named them in your
# config file, which you are free to change.
x0 <- data$x0
x1 <- data$x1
x2 <- data$x2
x3 <- data$x3
x4 <- data$x4
x5 <- data$x5
X <- c(x0, x1, x2, x3, x4, x5)

# This is where we actually run our "model".
# Here we are using a synthetic function called hartman6
# But you could substitute it for your own model in
# a number of ways.
res <- hartman6(X)

# In this case, we directly ran the model, so we are getting back a number
# or nan, so we know if it succeeded or failed. If you are submitting a job
# to an HPC (a super computer) queue, this might work, or you might have to
# rely on another method. Other options could be relying on log file output
# or information from querying the queue itself,
# though those may be better as stand alone `Set Trial Status Scripts`
if (!is.na(res)) {

    # if it was a success, we don't even need to write out trial status,
    # it is assumed a success if we write out data and don't fail
    out_data <- list(
        metric=res
        # trial_status=unbox("COMPLETED")  #  this is optional if it succeeds
    )

} else {

    # If we fail, then we do need to include a trial status, and mark it as failed.
    out_data <- list(
        trial_status=unbox("FAILED")
    )
}

json_data <- toJSON(out_data, pretty = TRUE)
write(json_data, file.path(trial_dir, "output.json"))

## Running our script

To run our script we just need to path the config file to BOA's CLI

```python
python -m boa --config-file path/to/config.yaml
```

or

```python
python -m boa -c path/to/config.yaml
```

In [7]:
output = !python -m boa -c {config_path}  # we capture ipython terminal output to python variable
o = "\n".join(ln for ln in output)  # it comes in as a list, we convert to string
o = o.replace(str(r_dir),  "[/path/to/your/dir/]")  # replace the actual dir with a stand in for privacy reasons
Code(o)

[WARNING 07-13 14:30:49] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 07-13 14:30:50] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x4', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x5', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 07-13 14:30:50] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-13 14:30:50] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=6 num_trials=None use_batch_trials=False
[INFO 07-13 14:30:50] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=12
[INFO 07-13 14:30:50] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=12
[INFO 07-13 14:30:50] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 12 trials, GPEI for subsequent trials]). Iterations after 12 will take longer to generate due to model-fitting.
[INFO 07-13 14:30:50] Scheduler: `Scheduler` requires experiment to have immutable search space and optimization config. Setting property immutable_search_space_and_opt_config to `True` on experiment.
[INFO 2023-07-13 14:30:50,865 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: [/path/to/your/dir/]/r_streamlined_run_20230713T143050
Start Time: 20230713T143050
Version: 0.8.7.dev4+gae30cf2.d20230713

##############################################

[INFO 07-13 14:30:50] Scheduler: Running trials [0]...
[INFO 07-13 14:30:52] Scheduler: Running trials [1]...
[INFO 07-13 14:30:54] Scheduler: Running trials [2]...
[INFO 07-13 14:30:56] Scheduler: Running trials [3]...
[INFO 07-13 14:30:57] Scheduler: Running trials [4]...
[INFO 07-13 14:30:59] Scheduler: Running trials [5]...
[INFO 07-13 14:31:00] Scheduler: Running trials [6]...
[INFO 07-13 14:31:01] Scheduler: Running trials [7]...
[INFO 07-13 14:31:04] Scheduler: Running trials [8]...
[INFO 07-13 14:31:07] Scheduler: Running trials [9]...
[INFO 07-13 14:31:08] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-13 14:31:08] Scheduler: Fetching data for trials: 0 - 9.
[INFO 2023-07-13 14:31:08,745 MainProcess] boa: Saved JSON-serialized state of optimization to `[/path/to/your/dir/]/r_streamlined_run_20230713T143050/scheduler.json`.
Boa version: 0.8.7.dev4+gae30cf2.d20230713
[INFO 2023-07-13 14:31:08,803 MainProcess] boa: Trials so far: 10
Running trials: 
Will Produce next trials from generation step: Sobol
Best trial so far: {9: {'metric': -0.5032}}
[INFO 07-13 14:31:08] Scheduler: Running trials [10]...
[INFO 07-13 14:31:09] Scheduler: Running trials [11]...
[INFO 07-13 14:31:15] Scheduler: Running trials [12]...
[INFO 07-13 14:31:16] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.
[INFO 07-13 14:31:21] Scheduler: Running trials [13]...
[INFO 07-13 14:31:22] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.
[INFO 07-13 14:31:24] Scheduler: Running trials [14]...
[INFO 07-13 14:31:26] Scheduler: Retrieved COMPLETED trials: 10 - 